In [1]:
import sys
sys.path.append("/home/ubuntu/ins-plt/DS-Inference-Scripts/rl_scripts/retrain/")

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from core import *
from eq_common_utils.utils.config.s3_config import s3_config

s3_helper = s3_config()
s3_client = s3_helper._s3Client
paginator = s3_client.get_paginator("list_objects_v2")

In [7]:
ssga_etfs_bucket = "eq-model-input"
ssga_etfs_prefix = "etf_holdings/SSGA/"
ssga_etfs = []

page_iterator = paginator.paginate(Bucket=ssga_etfs_bucket, Prefix=ssga_etfs_prefix, Delimiter="/")

for page in page_iterator:
    if "CommonPrefixes" in page:
        for common_prefix in page['CommonPrefixes']:
            ssga_etfs.append(common_prefix['Prefix'].split('/')[-2])

In [5]:
def get_features(prefix):
    key = s3_helper.get_latest_obj(ssga_etfs_bucket, ssga_prefix)['Key']
    data = s3_helper.read_as_dataframe(ssga_etfs_bucket, key)
    return data.columns.tolist()

def refresh_date(data):
    if ("Weight (%)" in data) and (data['Weight (%)'] > 0):
        if "Weight" in data.columns:
            data.drop(columns=['Weight'])
        data = data.rename(columns={"Weight (%)": "Weight"})
    return data

def refresh_data(prefix):
    page_iterator = paginator.paginate(Bucket=ssga_etfs_bucket, Prefix=prefix)
    for page in page_iterator:
        if "Contents" in page:
            for content in tqdm(page['Contents']):
                data = s3_helper.read_as_dataframe(ssga_etfs_bucket, content['Key'])
                data = refresh_date(data)
                s3_helper.write_advanced_as_df(data, ssga_etfs_bucket, content['Key'])

In [7]:
for ssga_etf in ssga_etfs:
    ssga_prefix = ssga_etfs_prefix + f"{ssga_etf}/"
    print(get_features(ssga_prefix))

['Name', 'Ticker', 'Identifier', 'Weight', 'Market Value', 'Shares Held']
['Name', 'Identifier', 'SEDOL', 'Weight', 'Coupon', 'Par Value', 'Market Value', 'Local Currency', 'Maturity']
['Name', 'Identifier', 'SEDOL', 'Weight', 'Coupon', 'Par Value', 'Market Value', 'Local Currency', 'Maturity']


['Name', 'Identifier', 'SEDOL', 'Weight', 'Coupon', 'Par Value', 'Market Value', 'Local Currency', 'Maturity', 'ISIN']
['Name', 'Identifier', 'SEDOL', 'Weight', 'Coupon', 'Par Value', 'Market Value', 'Local Currency', 'Maturity', 'ISIN']
['Name', 'Ticker', 'Identifier', 'SEDOL', 'Weight', 'Market Value', 'Notional Value', 'Sector', 'Shares Held', 'Local Currency']
['Name', 'Ticker', 'Identifier', 'SEDOL', 'Weight', 'Sector', 'Shares Held', 'Local Currency']
['Name', 'Identifier', 'SEDOL', 'Weight', 'Coupon', 'Par Value', 'Market Value', 'Local Currency', 'Maturity']
['Name', 'Ticker', 'Identifier', 'SEDOL', 'Weight', 'Sector', 'Shares Held', 'Local Currency']
['Name', 'Ticker', 'Identifier', 'SEDOL', 'Weight', 'Market Value', 'Notional Value', 'Sector', 'Shares Held', 'Local Currency', 'Number of Contracts']
['Name', 'Ticker', 'Identifier', 'SEDOL', 'Weight', 'Sector', 'Shares Held', 'Local Currency']
['Name', 'Ticker', 'Identifier', 'SEDOL', 'Weight', 'Sector', 'Shares Held', 'Local C

{'Key': 'etf_holdings/SSGA/BWX/2026-02-06.csv',
 'LastModified': datetime.datetime(2026, 2, 10, 4, 1, 42, tzinfo=tzlocal()),
 'ETag': '"4857a5f58bf256249d94b2e0dfddbd75"',
 'ChecksumAlgorithm': ['CRC32'],
 'Size': 171710,
 'StorageClass': 'STANDARD'}